In [39]:
import os
from os import listdir

import numpy as np

import matplotlib.pyplot as plt
from google.colab import drive
from keras import backend as K
from keras.callbacks import ModelCheckpoint
from keras.layers import (Conv2D, Conv2DTranspose, Dropout, Input,
                          MaxPooling2D, concatenate)
from keras.models import Model
from keras.optimizers import Adam
from matplotlib import pyplot as plt
from skimage.io import imread, imsave
from skimage.transform import resize
from sklearn.model_selection import train_test_split

drive.mount('/content/gdrive/')

data_path = '/content/gdrive/My Drive/train'


Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [0]:
def create_train_test_data():
    HEIGTH = 420
    WIDTH = 580
    """Create train and test data from dataset"""
    images_dir = os.listdir(data_path)
    num_images = int(len(images_dir))

    images = np.ndarray((num_images, HEIGTH, WIDTH), dtype=np.uint8)
    images_mask = np.ndarray((num_images, HEIGTH, WIDTH), dtype=np.uint8)

    i = 0
    for image_name in images_dir:
        if 'mask' in image_name:
            continue
        mask_name = image_name.split('.')[0] + '_mask.tif'
        if mask_name.find('(1)') == -1:
          image = imread(
              os.path.join(data_path, image_name),
              as_gray=True,
              plugin='matplotlib'
          )
          mask = imread(
              os.path.join(data_path, mask_name),
              as_gray=True,
              plugin='matplotlib'
          )

        image = np.array([image])
        mask = np.array([mask])

        images[i] = image
        images_mask[i] = mask
        
        i += 1

    train, test, mask_train, mask_test = train_test_split(
        images,
        images_mask,
        test_size=0.20,
        random_state=42
    )

    return train, mask_train, test, mask_test


In [0]:
HEIGHT = 96
WIDTH = 96
smooth = 1.

In [0]:
def dice_coef(y_true, y_pred):
    """Return dice coefficient"""
    y_true = K.flatten(y_true)
    y_pred = K.flatten(y_pred)
    intersec = K.sum(y_true * y_pred)
    return (2. * intersec + smooth) / (K.sum(y_true) + K.sum(y_pred) + smooth)


def dice_coef_loss(y_true, y_pred):
    """Get loss for dice coefficient"""
    return 1-dice_coef(y_true, y_pred)

def preprocess(imgs):
    """Reduce dimension of images"""
    imgs_p = np.ndarray((imgs.shape[0], HEIGHT, WIDTH), dtype=np.uint8)
    for i in range(imgs.shape[0]):
        imgs_p[i] = resize(imgs[i], (HEIGHT, WIDTH), preserve_range=True)

    imgs_p = imgs_p[..., np.newaxis]
    return imgs_p

In [0]:
def conv_net():
    """Compile convolutional network with dice coefficient metrics"""
    inputs = Input((HEIGHT, WIDTH, 1))
    conv1 = Conv2D(32, (3, 3), activation='elu', padding='same')(inputs)
    conv1 = Dropout(0.2)(conv1)
    conv1 = Conv2D(32, (3, 3), activation='elu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(64, (3, 3), activation='elu', padding='same')(pool1)
    conv2 = Dropout(0.2)(conv2)
    conv2 = Conv2D(64, (3, 3), activation='elu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(128, (3, 3), activation='elu', padding='same')(pool2)
    conv3 = Dropout(0.2)(conv3)
    conv3 = Conv2D(128, (3, 3), activation='elu', padding='same')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = Conv2D(256, (3, 3), activation='elu', padding='same')(pool3)
    conv4 = Dropout(0.2)(conv4)
    conv4 = Conv2D(256, (3, 3), activation='elu', padding='same')(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

    conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(pool4)
    conv5 = Dropout(0.2)(conv5)
    conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(conv5)

    up6 = concatenate([Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(conv5), conv4], axis=3)
    conv6 = Conv2D(256, (3, 3), activation='elu', padding='same')(up6)
    conv6 = Dropout(0.2)(conv6)
    conv6 = Conv2D(256, (3, 3), activation='elu', padding='same')(conv6)

    up7 = concatenate([Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv6), conv3], axis=3)
    conv7 = Conv2D(128, (3, 3), activation='elu', padding='same')(up7)
    conv7 = Dropout(0.2)(conv7)
    conv7 = Conv2D(128, (3, 3), activation='elu', padding='same')(conv7)

    up8 = concatenate([Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv7), conv2], axis=3)
    conv8 = Conv2D(64, (3, 3), activation='elu', padding='same')(up8)
    conv8 = Dropout(0.2)(conv8)
    conv8 = Conv2D(64, (3, 3), activation='elu', padding='same')(conv8)

    up9 = concatenate([Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(conv8), conv1], axis=3)
    conv9 = Conv2D(32, (3, 3), activation='elu', padding='same')(up9)
    conv9 = Dropout(0.4)(conv9)
    conv9 = Conv2D(32, (3, 3), activation='elu', padding='same')(conv9)

    conv10 = Conv2D(1, (1, 1), activation='softmax')(conv9)
  
    model.compile(optimizer=Adam(lr=1e-5), loss=dice_coef_loss, metrics=[dice_coef])


    return model

In [0]:
def model():
  """Fit model nerve net and get plot"""
  model = conv_net()
  checkpoint = ModelCheckpoint('checkpoint.h5', monitor='val_loss', save_best_only=True,verbose = 1)
  return model

In [0]:
def plot_loss(model):
  plt.plot(model.history['loss'],'r')
  plt.plot(model.history['val_loss'],'g')
  plt.rcParams['figure.figsize'] = (8, 6)
  plt.xlabel("epoch")
  plt.ylabel("loss")
  plt.legend(['train', 'validation'], loc='upper left')
  plt.savefig('model_loss.png')

In [0]:
def train():
  batch_size = 256
  epochs = 120
  
  train, mask_train, test, mask_test = create_train_test_data()

  train = preprocess(train)
  mask_train = preprocess(mask_train)
  train = train.astype('float32')
  mean = np.mean(train)
  std = np.std(train)  

  train -= mean
  train /= std

  mask_train = mask_train.astype('float32')
  mask_train /= 255.

  model1 = conv_net()
  model1.summary()
  checkpoint = ModelCheckpoint('checkpoint.h5', monitor='val_loss', save_best_only=True,verbose = 1)
  
  history = model1.fit(train, mask_train, batch_size=256, epochs=100, verbose=1, shuffle=True,
              validation_split=0.2,
              callbacks=[checkpoint])
  
 
  model.evaluate(test, mask_test)
    
  return history

In [0]:
history = train()
  
plot_loss(history)